#Import libraries

In [11]:
import openpyxl
import pandas as pd
import dask.dataframe as dd

#Define a function to read input data and create a dictionary

In [55]:
des = ['Existing PSP services and date of interaction of patient with each service',
        'App service data', 'Demographic data of patient', 'Data of campaign targeting patients to keep them engaged',
        'Data of patient calls with the support team to discuss, order and provide services',
        'Specialist team providing assistance and guidance/motivation for patient engagement with services',
        'Tracking the status of shared content with patient', 'Tracking the status of shared content with patient',
        'Drug sponsor of patient']

def read_data(inputfile):
    """
    This function helps create a data dictionary.
    Input: Workbook/spreadsheet
    Output: Unique patients, #rows, #columns, #duplicates, #NULLs in columns in each tab/sheet
    """
    
    wb = openpyxl.load_workbook(inputfile)
    names = wb.sheetnames
    n1 = len(names)
    print('Total number of sheets in the workbook = {}'.format(n1))
    print('Sheet names respectively:', names)
    print('')

    columns = [] #columns of data dictionary
    for i in range(1, n1):
        df = pd.read_excel(inputfile, sheet_name = names[i])
        n2 = len(df['patient_id'].unique())
        n0 = len(df[df.duplicated()])
        col_names = df.columns.tolist()
        cols = df.columns[df.isnull().any()].tolist()
        #n5 = len(cols)
            
        n3 = df.shape[0]
        n4 = df.shape[1]
        sheet = names[i]
        dt = [] #datatypes of dataframe columns
        for j in range(n4):
            dt.append(str(df[df.columns[j]].dtype))
                       
        #print('Number of unique patients in sheet {} = {}'.format(i, n2))
        #print('Number of duplicates in sheet {} = {}'.format(i, n0))
        #print('#Columns having missing values in sheet {} = {}'.format(i, n5))
    
        columns.append({'Sheet': sheet, 'Description': des[i], '#Duplicates': n0, '#Unique patients': n2, '#Rows': n3, '#Columns': n4, 'Column names':col_names, 'Datatypes of columns': dt, 'Columns with NULLs': cols})
    dict_df = pd.DataFrame(columns)
    dict_df.to_csv('Data dictionary.csv', index = False)

    return print('Dictionary created.')


In [13]:
#for i, value in enumerate(df0.dtypes):
#    print(value)
inputfile = 'AbbVie_Capstone.xlsx'

#OUTPUT

In [57]:
%%time
read_data(inputfile)

Total number of sheets in the workbook = 9
Sheet names respectively: ['Interaction', 'App', 'Demog', 'Campaign', 'Support_services', 'Specialist', 'Content Access 1', 'Content Access 2', 'Payor']

Dictionary created.
CPU times: total: 2min 46s
Wall time: 2min 48s


In [3]:
df0 = pd.read_excel(inputfile, sheet_name = 'Interaction')
print(len(df0['patient_id'].unique()), df0.shape)
print(df0['service'].unique(), len(df0[df0.duplicated()]))

2994 (489212, 3)
['Support_person' 'Collateral' 'App' 'Email' 'Moving bag' 'Med Rem'
 'Removal kit' 'training_gadget' 'HPS_low' 'HPP-high'] 43


In [4]:
df2 = pd.read_excel(inputfile, sheet_name = 'Demog')
df3 = pd.read_excel(inputfile, sheet_name = 'Payor')

In [7]:
df_1 = pd.merge(df0, df2, how = 'outer')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2528437 entries, 0 to 2528436
Data columns (total 7 columns):
 #   Column               Dtype         
---  ------               -----         
 0   patient_id           int64         
 1   service              object        
 2   date of interaction  datetime64[ns]
 3   enrolled_channel     object        
 4   enrolled date        datetime64[ns]
 5   gender               object        
 6   age                  object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 154.3+ MB


In [8]:
df_2 = pd.merge(df_1, df3, how = 'outer')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2540922 entries, 0 to 2540921
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   patient_id           int64         
 1   service              object        
 2   date of interaction  datetime64[ns]
 3   enrolled_channel     object        
 4   enrolled date        datetime64[ns]
 5   gender               object        
 6   age                  object        
 7   payor                object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 174.5+ MB


In [10]:
df_2.to_csv('Interaction+demography+payor.csv', index = False)

#Merging 2 sheets at a time (except sheet 0 - 'Interaction') sequentially

In [14]:
df1 = pd.read_excel(inputfile, sheet_name = 'App')
df2 = pd.read_excel(inputfile, sheet_name = 'Demog')
df3 = pd.read_excel(inputfile, sheet_name = 'Payor')

df4 = pd.read_excel(inputfile, sheet_name = 'Campaign')
df5 = pd.read_excel(inputfile, sheet_name = 'Support_services')
df6 = pd.read_excel(inputfile, sheet_name = 'Specialist')
df7 = pd.read_excel(inputfile, sheet_name = 'Content Access 1')
df8 = pd.read_excel(inputfile, sheet_name = 'Content Access 2')

In [ ]:
print(df7.info())

In [ ]:
#print(df7.columns.tolist())
#print(df8.columns.tolist())

print(df8.info())

In [ ]:
df77 = df7.drop(['last_click', 'last_open', 'total_clicks'], axis = 1) #last_click & last_open has most NULLs
df77.columns

In [ ]:
df88 = df8.drop(['total_clicks'], axis = 1) #this is in general higher than total_opens
df88.columns

In [ ]:
df12 = pd.merge(df1, df2, how = "outer") 
df123 = pd.merge(df12, df3, how = 'outer')
len(df123['patient_id'].unique()) #App plus demog plus payor 

In [ ]:
df123.info()

In [ ]:
df11 = df123.drop(['response_time'], axis = 1) #Not much info
df11.columns

In [ ]:
df45 =  pd.merge(df4, df5, how = "outer") 
len(df45['patient_id'].unique()) #campaign plus support services

In [ ]:
df45.info()

In [ ]:
df67 = pd.merge(df6, df77, how = "outer") 
df678 = pd.merge(df67, df88, how = "outer")
len(df678['patient_id'].unique()) #specialist plus content access

In [ ]:
df678.info()

In [ ]:
df22 = df678.drop(['document_views'], axis = 1) #Most NULLs
df22.columns

In [ ]:
#dff = dd.merge(df22, df11, how = 'outer')
#dff.info(memory_usage = 'deep')

df11.to_csv('App+Demography+Payor.csv', index = False)


In [ ]:
#df11.shape[0]+df22.shape[0]+df45.shape[0]

In [ ]:
#df11.memory_usage(index = False)
df11.isnull().any()

In [ ]:
#df22.memory_usage(index = False)
df22.isna().any()

In [ ]:
#df45.memory_usage(index = False)
df45.isnull().any()

In [ ]:
#prob = 0.95
#critical = chi2.ppf(prob, dof) #critical value
#print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
#if abs(stat) >= critical:
# print('Dependent (reject Null hypothesis)') #significant result
#else:
# print('Independent (accept Null hypothesis)')

In [ ]:
##Med Rem -> SMS, Phone call, Notification/Alert, Email

#df2.drop(df2[df2['delivery status'] == 'Left Message - Third Attempt Made; Did Not Hear From Patient'].index, 
#         inplace = True)
#df2.drop(df2[df2['delivery status'] == 'UNDELIVERED'].index, inplace = True)

In [ ]:
#df['payor'] = df['payor'].replace(['Commercial', 'Government', 'manufacturer', 'Unknown', 'Others',
       #'Commercial +', 'Government +'],[1, 2, 3, 4, 5, 6, 7])

#df['payor'].dtype

#df['gender'] = df['gender'].replace(['M', 'F', 'UNKNOWN'],[1, 2, 3])

#df['age'] = df['age'].replace(['26-40', '41-65', '65+', '18-25', '<18', 'UNKNOWN'],[1, 2, 3, 4, 5, 6])

#df['service'] = df['service'].replace(['Support_person', 'Collateral', 'App', 'Email', 'Moving bag',
       #'Med Rem', 'Removal kit', 'training_gadget', 'HPS_low', 'HPP-high'],[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
#nrows = 200000 
#df1 = pd.read_csv('App_Content Access_Campaign.csv', nrows = nrows, low_memory = False)

#df1['response_time'] =  pd.to_datetime(df1['response_time'], format='%H:%M:%S')
#df1['response_hour'] = df1['response_time'].dt.hour

#df1['Months on program'] = df1['Months on program'].astype(object)

#dff = df1.join(df1, df2, how = 'outer', on = "patient_id") 

#dff = dd.merge(df11, df22, how = 'outer') 

#ads[ads.select_dtypes(['object']).columns] = ads.select_dtypes(['object']).apply(lambda x: x.astype('category'))

##df.count()
#inputFile = "C:\\Users\\RanjaSarkar\\Desktop\\PSP\\Raw\\AbbVie_Capstone.xlsx"
#show(df)
#msn.bar(df, color = 'blue')
#len(df[df.duplicated(keep = 'first')])  


In [ ]:
alpha = 0.05 #95% probability that the Null is true
print('significance = %.4f, p = %.4f' % (alpha, p))
if p <= alpha:
 print('Dependent (reject Null hypothesis)') #significant result
else:
 print('Independent (accept Null hypothesis)')

In [ ]:
#df['payor'] = df['payor'].fillna('Unknown')
#df['payor'] = df['payor'].replace(['Cash','Cash/ICF'],'Others')

#df1['patient_id'] = df1['patient_id'].astype(np.int64)